In [6]:
!pip install transformers
!pip install scikit-learn


  Using cached transformers-4.32.1-py3-none-any.whl (7.5 MB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached certifi-2023.7.22-py3-none-any.whl (158 kB)


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\tqdm.exe' -> 'C:\\Python311\\Scripts\\tqdm.exe.deleteme'


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Python311\python.exe -m pip install --upgrade pip


#Training


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

excel_name = "Cleansed OT.agents.xlsx"
dataset = pd.read_excel(excel_name, sheet_name="Sheet1")

def generate_start_idx_col(dataset, label, context):
    for index, row in dataset.iterrows():
        context_col = context
        find_col = label
        start_idx = row[context_col].find(row[find_col])
        if start_idx <0:
          start_idx = 0
        end_idx = start_idx + len(row[find_col]) - 1
        dataset.at[index, "start_idx"] = int(start_idx)
        dataset.at[index, "end_idx"] = int(end_idx)


#get idx start and end
generate_start_idx_col(dataset, "name", "full name")
#add same question
question_prompt = "How do I address him?"
dataset["question"] = question_prompt
dataset.to_csv("check.csv")

# Split the dataset into train, validation, and test sets
train_df, test_df = train_test_split(dataset, test_size=0.1, random_state=42)
test_df, val_df = train_test_split(test_df, test_size=0.1, random_state=42)
train_contexts, train_questions, train_answers = train_df["full name"], train_df["question"], train_df[["name", "start_idx", "end_idx"]]
val_contexts, val_questions, val_answers = val_df["full name"], val_df["question"], val_df[["name", "start_idx", "end_idx"]]
test_contexts, test_questions, test_answers = test_df["full name"], test_df["question"], test_df[["name", "start_idx", "end_idx"]]

# Printing the size of each set to verify
print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")


from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#prep format to list
# Tokenize contexts and questions separately
train_encodings = tokenizer(list(train_contexts), list(train_questions), truncation=True, padding=True)
val_encodings = tokenizer(list(val_contexts), list(val_questions), truncation=True, padding=True)
test_encodings = tokenizer(list(test_contexts) , list(test_questions) , truncation=True, padding=True )

print(train_answers)
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []

    for i , tuple_row in enumerate(answers.iterrows()) :
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i,int(tuple_row[1]["start_idx"])))
        end_positions.append(encodings.char_to_token(i,int(tuple_row[1]["end_idx"])))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift one token forward
        go_back = 1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, int(tuple_row[1]["end_idx"]-go_back))
            go_back +=1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# Apply function to our data
add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)
add_token_positions(test_encodings, test_answers)




KeyboardInterrupt



In [ ]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)
test_dataset= SquadDataset(test_encodings)

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm

# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device
model.to(device)
# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=1e-6)

# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
number_of_epochs = 100
scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=0, num_training_steps=len(train_loader)*number_of_epochs)


for epoch in range(number_of_epochs):
    # set model to train mode
    model.train()
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
    scheduler.step()

# switch model out of training mode
model_path = 'name_extractor_model'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
model.eval()

#val_sampler = SequentialSampler(val_dataset)
val_loader = DataLoader(val_dataset, batch_size=16)


acc = []

# initialize loop for progress bar
loop = tqdm(val_loader)
# loop through batches
for batch in loop:
    # we don't need to calculate gradients as we're not training
    with torch.no_grad():
        # pull batched items from loader
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_true = batch['start_positions'].to(device)
        end_true = batch['end_positions'].to(device)
        # make predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        # pull preds out
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        # calculate accuracy for both and append to accuracy list
        acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
        acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
# calculate average accuracy in total
acc = sum(acc)/len(acc)
print("T/F\tstart\tend\n")
for i in range(len(start_true)):
    print(f"true\t{start_true[i]}\t{end_true[i]}\n"
          f"pred\t{start_pred[i]}\t{end_pred[i]}\n")

100%|██████████| 2/2 [00:00<00:00, 27.56it/s]

T/F	start	end

true	2	6
pred	2	6

true	9	11
pred	9	11

true	8	9
pred	8	9

true	8	9
pred	8	9

true	3	6
pred	3	6

true	8	8
pred	8	8

true	11	12
pred	11	12

true	3	7
pred	3	7



In [ ]:
!zip -r /content/model.zip /content/name_extractor_model


  adding: content/name_extractor_model/ (stored 0%)
  adding: content/name_extractor_model/tokenizer.json (deflated 72%)
  adding: content/name_extractor_model/merges.txt (deflated 53%)
  adding: content/name_extractor_model/tokenizer_config.json (deflated 80%)
  adding: content/name_extractor_model/pytorch_model.bin (deflated 8%)
  adding: content/name_extractor_model/special_tokens_map.json (deflated 85%)
  adding: content/name_extractor_model/config.json (deflated 49%)
  adding: content/name_extractor_model/vocab.json (deflated 59%)


#Download and Store Model

In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Copy the file to Google Drive
shutil.copy("/content/model.zip", "/content/drive/MyDrive/")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/model.zip'

#Load and Evaluate

In [22]:
import torch
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import pandas as pd

# Path to the directory where you saved the model and tokenizer
model_path = 'name_extractor_model'

# Load the pretrained model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Move the model to the appropriate device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Set the model to evaluation mode
model.eval()

# Initialize empty lists for storing predictions and ground truths
start_pred_list = []
end_pred_list = []
start_true_list = []
end_true_list = []
predicted_answers = []

model.eval()
print("Model Loaded")

# Print the predicted answers along with the true labels
predicted_answers = []
input_contexts= list(pd.read_excel("testing.xlsx", sheet_name = "raw").loc[:,"full name"])
questions = ["How do I address him?" for x in range(len(input_contexts))]
number_of_entries = len(input_contexts)
# Iterate through input contexts and questions
print(f"Processing {number_of_entries} entries")

# Progress
interval = int(number_of_entries/100)


for index,(context, question) in enumerate(zip(input_contexts, questions)):
    # Encode the input context and question
    inputs = tokenizer(context, question, return_tensors='pt', truncation=True, padding=True)
    inputs.to(device)

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    # Get the predicted answer span
    start_pos = torch.argmax(start_logits)
    end_pos = torch.argmax(end_logits) + 1  # Adding 1 to include the end position

    # Decode the predicted answer span using the tokenizer
    predicted_answer = tokenizer.decode(inputs['input_ids'][0][start_pos:end_pos], skip_special_tokens=True)
    predicted_answer = predicted_answer.title().strip()
    predicted_answers.append(predicted_answer)
    if index %interval == 0 and index!= 0:
        print(f"Progress is at {int(index/number_of_entries*100)}%")

print("Processing done, saving to excel")
temp_df = pd.DataFrame(zip(input_contexts,predicted_answers) , columns=["Full Name" , "Name"])
temp_df.to_excel("predicted.xlsx" , index=False)

Model Loaded
Processing 2301 entries
Progress is at 0%
Progress is at 1%
Progress is at 2%
Progress is at 3%
Progress is at 4%
Progress is at 5%
Progress is at 6%
Progress is at 7%
Progress is at 8%
Progress is at 9%
Progress is at 10%
Progress is at 11%
Progress is at 12%
Progress is at 13%
Progress is at 14%
Progress is at 15%
Progress is at 16%
Progress is at 17%
Progress is at 18%
Progress is at 19%
Progress is at 20%
Progress is at 21%
Progress is at 22%
Progress is at 23%
Progress is at 24%
Progress is at 25%
Progress is at 26%
Progress is at 27%
Progress is at 28%
Progress is at 29%
Progress is at 30%
Progress is at 31%
Progress is at 32%
Progress is at 33%
Progress is at 34%
Progress is at 35%
Progress is at 36%
Progress is at 37%
Progress is at 38%
Progress is at 39%
Progress is at 40%
Progress is at 41%
Progress is at 42%
Progress is at 43%
Progress is at 44%
Progress is at 45%
Progress is at 46%
Progress is at 47%
Progress is at 48%
Progress is at 49%
Progress is at 50%
Prog